# Assemble Data

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.ndimage import uniform_filter1d  
import re

In [2]:
pd.set_option('display.max_columns', None)  

### Start with plot features

In [3]:
# get elevation features
df = pd.read_pickle('../data/plot_elev_features.pkl')

### Compute vectors for aspect and slope, and some interaction terms

In [4]:
df

,plot_id,slope_mean,slope_min,slope_max,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,pro_curve_max,plan_curve_mean,plan_curve_min,plan_curve_max,aspect_mean,aspect_min,aspect_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha
0,0,7.116202,1.671869,12.657069,0.000475,-0.042193,0.036974,-0.000320,-0.043223,0.031502,0.000475,-0.042193,0.036974,168.628837,98.054443,287.259827,202.678589,227.649673,216.987355,17.714752,42.685837,32.023511,"POLYGON ((749515.552 5085017.582, 749593.102 5...",24.971085,3.774245e+04,3.774245
1,1,7.157215,2.141364,15.119311,-0.000136,-0.058244,0.040260,0.000042,-0.033721,0.047448,-0.000136,-0.058244,0.040260,169.726747,121.037415,221.336365,99.260628,120.873428,110.290033,-85.703209,-64.090408,-74.673799,"POLYGON ((749293.527 5084266.311, 749284.600 5...",21.612801,3.191569e+04,3.191569
2,2,11.803730,5.291763,19.373894,0.000031,-0.034337,0.036800,0.000070,-0.034964,0.027805,0.000031,-0.034337,0.036800,163.651144,142.062317,202.059097,105.712486,124.070328,115.284763,-79.251350,-60.893509,-69.679062,"POLYGON ((749683.152 5084433.659, 749663.238 5...",18.357841,1.040724e+04,1.040724
3,3,9.123407,1.320997,14.460165,0.000095,-0.031373,0.025294,0.000140,-0.028785,0.033410,0.000095,-0.031373,0.025294,165.897492,133.854797,192.743149,95.084496,109.766037,102.207162,-89.879341,-75.197800,-82.756664,"POLYGON ((749671.796 5084356.545, 749653.826 5...",14.681541,1.270753e+04,1.270753
4,4,6.518723,0.240493,15.043138,0.000038,-0.061499,0.064917,0.000613,-0.062695,0.050341,0.000038,-0.061499,0.064917,155.414602,73.587646,263.266449,130.967911,178.674332,152.926721,-53.995926,-6.289505,-32.037104,"POLYGON ((750026.827 5084777.385, 750031.798 5...",47.706421,1.270447e+05,12.704472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,3.481692,0.022498,13.487131,-0.000261,-0.040569,0.043431,-0.000353,-0.055958,0.054019,-0.000261,-0.040569,0.043431,229.785101,0.000000,359.847321,246.021927,261.757812,254.108077,61.058090,76.793976,69.144233,"POLYGON ((750175.532 5085581.273, 750081.328 5...",15.735886,4.624099e+04,4.624099
62,62,3.963347,0.045724,8.905317,-0.000188,-0.043597,0.056039,-0.000059,-0.045036,0.047512,-0.000188,-0.043597,0.056039,209.739258,0.000000,359.976868,230.808212,253.110870,243.790732,45.844376,68.147034,58.826921,"POLYGON ((749819.022 5085646.705, 749476.313 5...",22.302658,9.619891e+04,9.619891
63,63,3.339648,0.010571,10.322133,-0.000130,-0.065293,0.073079,-0.000182,-0.057799,0.055700,-0.000130,-0.065293,0.073079,185.393326,0.031708,359.372559,218.849838,241.400696,231.685729,33.886002,56.436859,46.721879,"POLYGON ((749602.788 5085360.138, 749606.595 5...",22.550858,9.743973e+04,9.743973
64,64,5.642474,1.204210,11.197914,-0.000124,-0.058471,0.036657,0.000034,-0.061559,0.065094,-0.000124,-0.058471,0.036657,179.611351,110.309143,265.084839,213.873154,236.760712,227.058703,28.909317,51.796875,42.094850,"POLYGON ((749923.312 5085237.014, 749927.662 5...",22.887558,4.494281e+04,4.494281


## Now add NDVI for each plot to features

In [5]:
# Aspect: convert to radians and compute sin/cos
df['aspect_min_cos'] = np.cos(np.radians(df['aspect_min']))
df['aspect_min_sin'] = np.sin(np.radians(df['aspect_min']))

df['aspect_max_cos'] = np.cos(np.radians(df['aspect_max']))
df['aspect_max_sin'] = np.sin(np.radians(df['aspect_max']))

df['aspect_mean_cos'] = np.cos(np.radians(df['aspect_mean']))
df['aspect_mean_sin'] = np.sin(np.radians(df['aspect_mean']))

# Drop raw aspect values

df['slope_rad_mean'] = np.radians(df['slope_mean'])
# df['slope_rad_min'] = np.radians(df['slope_min'])
# df['slope_rad_max'] = np.radians(df['slope_max'])

df['slope_rad_mean_x'] = np.tan(df['slope_rad_mean']) * df['aspect_mean_cos']
df['slope_rad_mean_y'] = np.tan(df['slope_rad_mean']) * df['aspect_mean_sin']


# Interactions using slope (in radians) and curvature
df['slope_x_curve'] = df['slope_rad_mean_x'] * df['curve_mean']
df['slope_y_curve'] = df['slope_rad_mean_y'] * df['curve_mean']



### open up the filtered and smoothed ndvi df

In [6]:
veg_agg = pd.read_pickle('../data/ndvi/plots/final_df.pkl')

In [7]:
# veg_agg = veg_agg.dropna()
veg_agg

,plot_id,year,evi_smooth_mean_28,evi_smooth_mean_29,evi_smooth_mean_30,evi_smooth_mean_31,evi_smooth_mean_32,evi_smooth_mean_33,evi_smooth_mean_34,evi_smooth_mean_35,evi_smooth_mean_36,evi_smooth_mean_37,evi_smooth_mean_38,evi_smooth_mean_39,evi_smooth_mean_40,evi_smooth_mean_41,evi_smooth_mean_42,evi_smooth_mean_43,evi_smooth_mean_44,evi_smooth_slope_28,evi_smooth_slope_29,evi_smooth_slope_30,evi_smooth_slope_31,evi_smooth_slope_32,evi_smooth_slope_33,evi_smooth_slope_34,evi_smooth_slope_35,evi_smooth_slope_36,evi_smooth_slope_37,evi_smooth_slope_38,evi_smooth_slope_39,evi_smooth_slope_40,evi_smooth_slope_41,evi_smooth_slope_42,evi_smooth_slope_43,evi_smooth_slope_44,evi_smooth_std_28,evi_smooth_std_29,evi_smooth_std_30,evi_smooth_std_31,evi_smooth_std_32,evi_smooth_std_33,evi_smooth_std_34,evi_smooth_std_35,evi_smooth_std_36,evi_smooth_std_37,evi_smooth_std_38,evi_smooth_std_39,evi_smooth_std_40,evi_smooth_std_41,evi_smooth_std_42,evi_smooth_std_43,evi_smooth_std_44,mcari2_smooth_mean_28,mcari2_smooth_mean_29,mcari2_smooth_mean_30,mcari2_smooth_mean_31,mcari2_smooth_mean_32,mcari2_smooth_mean_33,mcari2_smooth_mean_34,mcari2_smooth_mean_35,mcari2_smooth_mean_36,mcari2_smooth_mean_37,mcari2_smooth_mean_38,mcari2_smooth_mean_39,mcari2_smooth_mean_40,mcari2_smooth_mean_41,mcari2_smooth_mean_42,mcari2_smooth_mean_43,mcari2_smooth_mean_44,mcari2_smooth_slope_28,mcari2_smooth_slope_29,mcari2_smooth_slope_30,mcari2_smooth_slope_31,mcari2_smooth_slope_32,mcari2_smooth_slope_33,mcari2_smooth_slope_34,mcari2_smooth_slope_35,mcari2_smooth_slope_36,mcari2_smooth_slope_37,mcari2_smooth_slope_38,mcari2_smooth_slope_39,mcari2_smooth_slope_40,mcari2_smooth_slope_41,mcari2_smooth_slope_42,mcari2_smooth_slope_43,mcari2_smooth_slope_44,mcari2_smooth_std_28,mcari2_smooth_std_29,mcari2_smooth_std_30,mcari2_smooth_std_31,mcari2_smooth_std_32,mcari2_smooth_std_33,mcari2_smooth_std_34,mcari2_smooth_std_35,mcari2_smooth_std_36,mcari2_smooth_std_37,mcari2_smooth_std_38,mcari2_smooth_std_39,mcari2_smooth_std_40,mcari2_smooth_std_41,mcari2_smooth_std_42,mcari2_smooth_std_43,mcari2_smooth_std_44,ndvi_smooth_mean_28,ndvi_smooth_mean_29,ndvi_smooth_mean_30,ndvi_smooth_mean_31,ndvi_smooth_mean_32,ndvi_smooth_mean_33,ndvi_smooth_mean_34,ndvi_smooth_mean_35,ndvi_smooth_mean_36,ndvi_smooth_mean_37,ndvi_smooth_mean_38,ndvi_smooth_mean_39,ndvi_smooth_mean_40,ndvi_smooth_mean_41,ndvi_smooth_mean_42,ndvi_smooth_mean_43,ndvi_smooth_mean_44,ndvi_smooth_slope_28,ndvi_smooth_slope_29,ndvi_smooth_slope_30,ndvi_smooth_slope_31,ndvi_smooth_slope_32,ndvi_smooth_slope_33,ndvi_smooth_slope_34,ndvi_smooth_slope_35,ndvi_smooth_slope_36,ndvi_smooth_slope_37,ndvi_smooth_slope_38,ndvi_smooth_slope_39,ndvi_smooth_slope_40,ndvi_smooth_slope_41,ndvi_smooth_slope_42,ndvi_smooth_slope_43,ndvi_smooth_slope_44,ndvi_smooth_std_28,ndvi_smooth_std_29,ndvi_smooth_std_30,ndvi_smooth_std_31,ndvi_smooth_std_32,ndvi_smooth_std_33,ndvi_smooth_std_34,ndvi_smooth_std_35,ndvi_smooth_std_36,ndvi_smooth_std_37,ndvi_smooth_std_38,ndvi_smooth_std_39,ndvi_smooth_std_40,ndvi_smooth_std_41,ndvi_smooth_std_42,ndvi_smooth_std_43,ndvi_smooth_std_44,ndwi_smooth_mean_28,ndwi_smooth_mean_29,ndwi_smooth_mean_30,ndwi_smooth_mean_31,ndwi_smooth_mean_32,ndwi_smooth_mean_33,ndwi_smooth_mean_34,ndwi_smooth_mean_35,ndwi_smooth_mean_36,ndwi_smooth_mean_37,ndwi_smooth_mean_38,ndwi_smooth_mean_39,ndwi_smooth_mean_40,ndwi_smooth_mean_41,ndwi_smooth_mean_42,ndwi_smooth_mean_43,ndwi_smooth_mean_44,ndwi_smooth_slope_28,ndwi_smooth_slope_29,ndwi_smooth_slope_30,ndwi_smooth_slope_31,ndwi_smooth_slope_32,ndwi_smooth_slope_33,ndwi_smooth_slope_34,ndwi_smooth_slope_35,ndwi_smooth_slope_36,ndwi_smooth_slope_37,ndwi_smooth_slope_38,ndwi_smooth_slope_39,ndwi_smooth_slope_40,ndwi_smooth_slope_41,ndwi_smooth_slope_42,ndwi_smooth_slope_43,ndwi_smooth_slope_44,ndwi_smooth_std_28,ndwi_smooth_std_29,ndwi_smooth_std_30,ndwi_smooth_std_31,ndwi_smooth_std_32,ndwi_smooth_std_33,ndwi_smooth_std_34,ndwi_smooth_std_35,ndwi_smooth_std_36,ndwi_sm

In [8]:
df = df.merge(veg_agg, how = 'inner', on = 'plot_id')

### Now let's add some weather information

First load the wather that has been unzipped and clipped to the vineyard

In [9]:
weather = pd.read_pickle('../data/PRISM/df.pkl')



weather = (
    weather
    .groupby("date", as_index=False)
    .first()
)


weather['date'] = pd.to_datetime(weather['date'])
weather['doy'] = weather['date'].dt.dayofyear
weather['year'] = weather['date'].dt.year

weather

,date,n,ppt,tmax,tmean,tmin,vpdmax,vpdmin,doy,year
0,2016-01-01,1,0.0000,-0.415000,-3.153000,-5.8920,2.192000,0.4860,1,2016
1,2016-01-02,1,0.0000,-4.138000,-5.380000,-6.6220,1.168000,0.5970,2,2016
2,2016-01-03,1,0.0000,-4.943000,-6.424000,-7.9050,1.166000,0.5900,3,2016
3,2016-01-04,1,1.0160,-4.861000,-6.448000,-8.0370,0.841000,0.2900,4,2016
4,2016-01-05,1,0.0000,-2.711000,-4.315000,-5.9210,1.130000,0.2980,5,2016
...,...,...,...,...,...,...,...,...,...,...
3550,2025-09-20,1,0.0000,30.182299,21.725849,13.2694,26.377600,0.8048,263,2025
3551,2025-09-21,1,0.0000,32.516800,23.064949,13.6131,41.111298,2.2150,264,2025
3552,2025-09-22,1,1.5994,22.928999,15.978449,9.0279,14.408899,0.3890,265,2025
3553,2025-09-23,1,0.0000,22.489500,15.037100,7.5847,15.716000,0.2265,266,2025


### Crack at some frost and growing degree days cumulative per month

In [10]:
# Compute frost days and GDD
weather['frost'] = (weather['tmin'] < 0)
weather['gdd'] = (weather['tmean'] - 10).clip(lower=0) 

# disregard 2025 for now
weather = weather[weather['year'] != 2025].copy()

In [11]:
# Compute cumulative GDD for each month
weather['week'] = weather['date'].dt.week
weather['month'] = weather['date'].dt.month
cumulative_gdd = weather.groupby([ 'year', 'week'])['gdd'].sum().groupby(level=[0,1]).cumsum()
# Reset index to turn MultiIndex into columns
cumulative_gdd = cumulative_gdd.rename('cumulative_gdd').reset_index()

# Merge back to weather
weather = weather.merge(
    cumulative_gdd,
    on=['year', 'week'],
    how='left'
)


weather = weather[weather['week'].isin([str(i) for i in range(14,36)])]

In [12]:
# weather = weather[weather['week'].isin([str(i) for i in range(28,44)])]

In [13]:
weather.isna().sum()

date              0
n                 0
ppt               0
tmax              0
tmean             0
tmin              0
vpdmax            0
vpdmin            0
doy               0
year              0
frost             0
gdd               0
week              0
month             0
cumulative_gdd    0
dtype: int64

In [14]:
season_months = list(range(4,8))  # April through July
season_weather = weather[weather['month'].isin(season_months)]

season_summary = (
    season_weather.groupby('year')
    .agg(
        tmean_season_mean=('tmean','mean'),
        tmean_season_min=('tmean','min'),
        tmean_season_max=('tmean','max'),
        tmean_season_std=('tmean','std'),
        ppt_season_total=('ppt','sum'),
        gdd_season_total=('gdd','sum'),
        vpdmax_season_mean=('vpdmax','mean')
    )
    .reset_index()
)


In [15]:
season_summary

,year,tmean_season_mean,tmean_season_min,tmean_season_max,tmean_season_std,ppt_season_total,gdd_season_total,vpdmax_season_mean
0,2016,18.887060,10.159000,28.642002,4.437476,45.980000,1057.560111,26.954992
1,2017,17.871467,7.240000,28.394001,6.004024,49.992000,960.492095,26.625367
2,2018,18.548406,5.372000,28.832001,5.459486,50.814000,1050.342096,27.840240
3,2019,17.945773,8.441999,26.798899,4.722562,52.833000,973.636147,24.267951
4,2020,17.699510,4.772900,31.072498,5.321663,38.490000,963.108153,25.731328
5,2021,19.480037,5.284300,34.620598,6.626458,15.073600,1135.832433,32.327692
6,2022,16.564969,2.615000,31.220200,6.883147,91.932798,842.548455,23.136677
7,2023,18.580298,5.476100,28.088699,6.258424,25.257799,1069.879044,28.016618
8,2024,17.440196,5.854000,29.294098,6.076871,38.070699,925.366104,26.244144


In [16]:
weekly_months = list(range(7,9))  # July & August
weekly_weather = weather[weather['month'].isin(weekly_months)]

weekly_summary = (
    weekly_weather.groupby(['year','week'])
    .agg(
        tmean_week_mean=('tmean','mean'),
        tmax_week_max=('tmax','max'),
        tmin_week_min=('tmin','min'),
        ppt_week_sum=('ppt','sum'),
        gdd_week_sum=('gdd','sum'),
        vpdmax_week_sum=('vpdmax','sum')
    )
    .reset_index()
)

# Pivot to wide so each week becomes a set of features
weekly_summary_wide = weekly_summary.pivot(index='year', columns='week')
weekly_summary_wide.columns = [f'{var}_{wk}' for var,wk in weekly_summary_wide.columns]
weekly_summary_wide = weekly_summary_wide.reset_index()


In [17]:
weekly_summary_wide = weekly_summary_wide.fillna(0)

In [18]:
weather_summary = weekly_summary_wide.merge(season_summary, how = 'inner', on = 'year')

In [19]:
weather_summary

,year,tmean_week_mean_26,tmean_week_mean_27,tmean_week_mean_28,tmean_week_mean_29,tmean_week_mean_30,tmean_week_mean_31,tmean_week_mean_32,tmean_week_mean_33,tmean_week_mean_34,tmean_week_mean_35,tmax_week_max_26,tmax_week_max_27,tmax_week_max_28,tmax_week_max_29,tmax_week_max_30,tmax_week_max_31,tmax_week_max_32,tmax_week_max_33,tmax_week_max_34,tmax_week_max_35,tmin_week_min_26,tmin_week_min_27,tmin_week_min_28,tmin_week_min_29,tmin_week_min_30,tmin_week_min_31,tmin_week_min_32,tmin_week_min_33,tmin_week_min_34,tmin_week_min_35,ppt_week_sum_26,ppt_week_sum_27,ppt_week_sum_28,ppt_week_sum_29,ppt_week_sum_30,ppt_week_sum_31,ppt_week_sum_32,ppt_week_sum_33,ppt_week_sum_34,ppt_week_sum_35,gdd_week_sum_26,gdd_week_sum_27,gdd_week_sum_28,gdd_week_sum_29,gdd_week_sum_30,gdd_week_sum_31,gdd_week_sum_32,gdd_week_sum_33,gdd_week_sum_34,gdd_week_sum_35,vpdmax_week_sum_26,vpdmax_week_sum_27,vpdmax_week_sum_28,vpdmax_week_sum_29,vpdmax_week_sum_30,vpdmax_week_sum_31,vpdmax_week_sum_32,vpdmax_week_sum_33,vpdmax_week_sum_34,vpdmax_week_sum_35,tmean_season_mean,tmean_season_min,tmean_season_max,tmean_season_std,ppt_season_total,gdd_season_total,vpdmax_season_mean
0,2016,25.205002,21.171429,21.465430,22.332858,26.459287,22.403715,22.534715,25.704573,22.136715,22.660335,34.245998,29.282000,33.933998,36.145000,38.367001,35.361000,36.619999,36.757000,34.264999,32.931000,16.2390,12.668000,13.613000,13.3720,13.302000,12.517000,13.460000,13.329000,11.484000,14.495000,0.0000,8.664,0.416,1.3400,0.000,0.0,1.6360,0.0000,0.0000,0.000,45.615005,78.200006,80.258007,86.330009,115.215010,86.826008,87.743006,109.932009,84.957008,37.981005,123.666996,190.313002,212.498001,217.765003,325.330994,251.961996,241.617001,322.193001,245.950994,99.841000,18.887060,10.159000,28.642002,4.437476,45.980000,1057.560111,26.954992
1,2017,23.972501,25.909287,24.611287,22.399144,25.903144,27.000430,27.344001,21.954144,21.872573,24.700751,33.455002,38.617001,34.581001,34.411999,38.254002,38.081001,38.814999,33.997002,32.930000,35.689999,13.6850,14.323000,13.960000,11.0010,14.709000,15.956000,16.559999,11.318000,10.597000,12.072000,0.0000,0.000,0.000,0.0000,0.000,0.0,0.0000,2.5440,0.0000,0.000,27.945002,111.365007,102.279007,86.794006,111.322010,119.003012,121.408010,83.679008,83.108009,58.803005,81.726997,337.918003,291.830996,261.624001,338.941002,362.042999,341.190994,225.084000,221.519001,170.049000,17.871467,7.240000,28.394001,6.004024,49.992000,960.492095,26.625367
2,2018,22.569000,21.481287,24.537287,24.634144,26.210001,25.791716,26.438001,23.559430,21.511572,19.840601,30.813999,35.476002,39.570000,38.115002,37.171001,39.797001,39.551998,36.111000,32.799000,32.681000,14.3250,10.587000,13.053000,12.1540,13.477000,12.524000,14.212000,12.941000,11.071000,9.873000,0.0000,0.000,0.000,0.0000,0.000,0.0,0.0000,0.0000,0.0000,0.544,12.569000,80.369007,101.761007,102.439007,113.470009,110.542009,115.066008,94.916010,80.581005,49.203007,33.625000,239.958996,308.595995,318.309998,344.541000,326.609001,341.502998,265.216995,206.999998,136.456999,18.548406,5.372000,28.832001,5.459486,50.814000,1050.342096,27.840240
3,2019,0.000000,21.963585,22.976742,21.657585,23.522957,23.840128,25.581885,22.707014,22.841228,23.053716,0.000000,32.497002,33.301998,32.126999,36.487000,35.709000,39.064999,32.546001,34.202000,34.187000,0.0000,13.393000,13.161000,10.8260,11.476000,12.883000,14.752000,13.414000,12.035000,11.879000,0.0000,0.000,0.000,0.0030,1.184,0.0,1.0880,3.0140,2.2280,0.000,0.000000,83.745098,90.837194,81.603096,94.660696,96.880899,109.073196,88.949095,89.888596,78.322298,0.000000,215.507000,214.587999,204.767002,286.650995,268.428001,284.464003,217.998001,228.601000,201.910999,17.945773,8.441999,26.798899,4.722562,52.833000,973.636147,24.267951
4,2020,0.000000,19.595740,21.684814,22.521785,24.700956,27.844042,23.199399,22.407357,26.311086,22.192871,0.000000,29.735001,33.486000,35.125999,38.930000,41.867001,35.421001,36.230000,38.446999,32.664001,0.0000,12.130000,12.623000,11.3460,12.055

In [20]:
# # 1️⃣ Make sure both have a 'year' column
# assert 'year' in df.columns
# assert 'year' in weather_summary.columns

# # 2️⃣ Merge weather summary into the plot-level df by year
# df = df.merge(weather_summary, on='year', how='left')

# # 3️⃣ Now multiply rainfall columns by profile curvature
# rain_cols = [c for c in df_merged.columns if 'ppt_week_sum' in c]

# for col in rain_cols:
#     df[f'{col}_x_pro_curve_mean'] = df[col] * df['pro_curve_mean']

In [21]:
# Merge
df = df.merge(weather_summary, on='year', how='left')

rain_cols = [c for c in df.columns if 'ppt_week_sum' in c]
topo_terms = ['slope_rad_mean_x', 'slope_rad_mean_y', 'pro_curve_mean', 'plan_curve_mean']

for rain_col in rain_cols:
    for topo_col in topo_terms:
        inter_col = f'{rain_col}_{topo_col}'
        df[inter_col] = df[rain_col] * df[topo_col]


In [22]:
df

,plot_id,slope_mean,slope_min,slope_max,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,pro_curve_max,plan_curve_mean,plan_curve_min,plan_curve_max,aspect_mean,aspect_min,aspect_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha,aspect_min_cos,aspect_min_sin,aspect_max_cos,aspect_max_sin,aspect_mean_cos,aspect_mean_sin,slope_rad_mean,slope_rad_mean_x,slope_rad_mean_y,slope_x_curve,slope_y_curve,year,evi_smooth_mean_28,evi_smooth_mean_29,evi_smooth_mean_30,evi_smooth_mean_31,evi_smooth_mean_32,evi_smooth_mean_33,evi_smooth_mean_34,evi_smooth_mean_35,evi_smooth_mean_36,evi_smooth_mean_37,evi_smooth_mean_38,evi_smooth_mean_39,evi_smooth_mean_40,evi_smooth_mean_41,evi_smooth_mean_42,evi_smooth_mean_43,evi_smooth_mean_44,evi_smooth_slope_28,evi_smooth_slope_29,evi_smooth_slope_30,evi_smooth_slope_31,evi_smooth_slope_32,evi_smooth_slope_33,evi_smooth_slope_34,evi_smooth_slope_35,evi_smooth_slope_36,evi_smooth_slope_37,evi_smooth_slope_38,evi_smooth_slope_39,evi_smooth_slope_40,evi_smooth_slope_41,evi_smooth_slope_42,evi_smooth_slope_43,evi_smooth_slope_44,evi_smooth_std_28,evi_smooth_std_29,evi_smooth_std_30,evi_smooth_std_31,evi_smooth_std_32,evi_smooth_std_33,evi_smooth_std_34,evi_smooth_std_35,evi_smooth_std_36,evi_smooth_std_37,evi_smooth_std_38,evi_smooth_std_39,evi_smooth_std_40,evi_smooth_std_41,evi_smooth_std_42,evi_smooth_std_43,evi_smooth_std_44,mcari2_smooth_mean_28,mcari2_smooth_mean_29,mcari2_smooth_mean_30,mcari2_smooth_mean_31,mcari2_smooth_mean_32,mcari2_smooth_mean_33,mcari2_smooth_mean_34,mcari2_smooth_mean_35,mcari2_smooth_mean_36,mcari2_smooth_mean_37,mcari2_smooth_mean_38,mcari2_smooth_mean_39,mcari2_smooth_mean_40,mcari2_smooth_mean_41,mcari2_smooth_mean_42,mcari2_smooth_mean_43,mcari2_smooth_mean_44,mcari2_smooth_slope_28,mcari2_smooth_slope_29,mcari2_smooth_slope_30,mcari2_smooth_slope_31,mcari2_smooth_slope_32,mcari2_smooth_slope_33,mcari2_smooth_slope_34,mcari2_smooth_slope_35,mcari2_smooth_slope_36,mcari2_smooth_slope_37,mcari2_smooth_slope_38,mcari2_smooth_slope_39,mcari2_smooth_slope_40,mcari2_smooth_slope_41,mcari2_smooth_slope_42,mcari2_smooth_slope_43,mcari2_smooth_slope_44,mcari2_smooth_std_28,mcari2_smooth_std_29,mcari2_smooth_std_30,mcari2_smooth_std_31,mcari2_smooth_std_32,mcari2_smooth_std_33,mcari2_smooth_std_34,mcari2_smooth_std_35,mcari2_smooth_std_36,mcari2_smooth_std_37,mcari2_smooth_std_38,mcari2_smooth_std_39,mcari2_smooth_std_40,mcari2_smooth_std_41,mcari2_smooth_std_42,mcari2_smooth_std_43,mcari2_smooth_std_44,ndvi_smooth_mean_28,ndvi_smooth_mean_29,ndvi_smooth_mean_30,ndvi_smooth_mean_31,ndvi_smooth_mean_32,ndvi_smooth_mean_33,ndvi_smooth_mean_34,ndvi_smooth_mean_35,ndvi_smooth_mean_36,ndvi_smooth_mean_37,ndvi_smooth_mean_38,ndvi_smooth_mean_39,ndvi_smooth_mean_40,ndvi_smooth_mean_41,ndvi_smooth_mean_42,ndvi_smooth_mean_43,ndvi_smooth_mean_44,ndvi_smooth_slope_28,ndvi_smooth_slope_29,ndvi_smooth_slope_30,ndvi_smooth_slope_31,ndvi_smooth_slope_32,ndvi_smooth_slope_33,ndvi_smooth_slope_34,ndvi_smooth_slope_35,ndvi_smooth_slope_36,ndvi_smooth_slope_37,ndvi_smooth_slope_38,ndvi_smooth_slope_39,ndvi_smooth_slope_40,ndvi_smooth_slope_41,ndvi_smooth_slope_42,ndvi_smooth_slope_43,ndvi_smooth_slope_44,ndvi_smooth_std_28,ndvi_smooth_std_29,ndvi_smooth_std_30,ndvi_smooth_std_31,ndvi_smooth_std_32,ndvi_smooth_std_33,ndvi_smooth_std_34,ndvi_smooth_std_35,ndvi_smooth_std_36,ndvi_smooth_std_37,ndvi_smooth_std_38,ndvi_smooth_std_39,ndvi_smooth_std_40,ndvi_smooth_std_41,ndvi_smooth_std_42,ndvi_smooth_std_43,ndvi_smooth_std_44,ndwi_smooth_mean_28,ndwi_smooth_mean_29,ndwi_smooth_mean_30,ndwi_smooth_mean_31,ndwi_smooth_mean_32,ndwi_smooth_mean_33,ndwi_smooth_mean_34,ndwi_smooth_mean_35,ndwi_smooth_mean_36,ndwi_smooth_mean_37,ndwi_smooth_mean_38,ndwi_smooth_mean_39,ndwi_smooth_mean_40,ndwi_smooth_mean_41,ndwi_smooth_mean_42,ndwi_smooth_mean_43,ndwi_smooth_mean_44,ndwi_smooth_slope_28,ndwi_smooth_slope_29,ndwi_smooth_slope_30,ndwi_smooth_

In [23]:
# sum_rain_cols = [col for col in weather_summary.columns if 'ppt' in col]

# sum_rain_cols

# interaction_cols = 

# Interaction between rainfall and profile curvature
# df['ppt_pro_curve'] = df['ppt_week_sum_{n}'] * df['pro_curve_mean']
df = df.drop(columns=['aspect_min', 'aspect_max', 'aspect_mean'])

# Drop redundant slope columns (keep the radian-based ones)
df = df.drop(columns=['slope_mean', 'slope_max', 'slope_min'], errors='ignore')
    
df = df.drop(columns = ['curve_mean','curve_min','curve_max', 'pro_curve_mean','pro_curve_min','pro_curve_max','plan_curve_min','plan_curve_mean','plan_curve_max'])

### Now we can combine data.

In [24]:
# df

In [25]:
# df['year'] = df['year'].astype(int)
# df = pd.merge(df, weather_summary, how = 'inner', on = 'year')

In [26]:
veg_indices = ['evi','mcari2','ndvi','rendvi','savi','ndwi']

keep_cols = [
    col for col in df.columns
    if not any(vi in col for vi in veg_indices)
]


In [27]:
keep_cols.remove('geometry')

In [28]:
keep_cols

['plot_id',
 'elev_min',
 'elev_max',
 'elev_mean',
 'elev_dev_min',
 'elev_dev_max',
 'elev_dev_mean',
 'total_relief',
 'area_m2',
 'area_ha',
 'aspect_min_cos',
 'aspect_min_sin',
 'aspect_max_cos',
 'aspect_max_sin',
 'aspect_mean_cos',
 'aspect_mean_sin',
 'slope_rad_mean',
 'slope_rad_mean_x',
 'slope_rad_mean_y',
 'slope_x_curve',
 'slope_y_curve',
 'year',
 'tmean_week_mean_26',
 'tmean_week_mean_27',
 'tmean_week_mean_28',
 'tmean_week_mean_29',
 'tmean_week_mean_30',
 'tmean_week_mean_31',
 'tmean_week_mean_32',
 'tmean_week_mean_33',
 'tmean_week_mean_34',
 'tmean_week_mean_35',
 'tmax_week_max_26',
 'tmax_week_max_27',
 'tmax_week_max_28',
 'tmax_week_max_29',
 'tmax_week_max_30',
 'tmax_week_max_31',
 'tmax_week_max_32',
 'tmax_week_max_33',
 'tmax_week_max_34',
 'tmax_week_max_35',
 'tmin_week_min_26',
 'tmin_week_min_27',
 'tmin_week_min_28',
 'tmin_week_min_29',
 'tmin_week_min_30',
 'tmin_week_min_31',
 'tmin_week_min_32',
 'tmin_week_min_33',
 'tmin_week_min_34',
 'tm

In [29]:
# df = df.drop(columns = [col for col in df.columns if 'std' in col])

In [30]:
df_ndvi_rendvi = pd.DataFrame(df[[col for col in df.columns if 'ndvi' in col] + keep_cols].copy())
df_ndvi_rendvi.to_pickle('../data/df_ndvi_rendvi.pkl')

In [31]:
df_evi = pd.DataFrame(df[[col for col in df.columns if 'evi' in col] + keep_cols].copy())
df_evi.to_pickle('../data/df_evi.pkl')

In [32]:
df_mcari2 = pd.DataFrame(df[[col for col in df.columns if 'mcari2' in col] + keep_cols].copy())
df_mcari2.to_pickle('../data/df_mcari2.pkl')

In [33]:
df_savi = pd.DataFrame(df[[col for col in df.columns if 'savi' in col] + keep_cols].copy())
df_savi.to_pickle('../data/df_savi.pkl')

In [34]:
df_ndwi = pd.DataFrame(df[[col for col in df.columns if 'ndwi' in col] + keep_cols].copy())
df_ndwi.to_pickle('../data/df_ndwi.pkl')

In [35]:
df.to_pickle('../data/df.pkl')